# Normalizacion de Registos de Pozo

## Configuracion del Entorno de trabajo

In [4]:
# Cargamos las librerias para cargar los datos al entorno de trabajo
import pandas as pd
import sys
import os

# Funciones importadas como modulos
notebook_dir = os.getcwd()
scripts_dir = os.path.join(notebook_dir, "..", "src")
sys.path.append(scripts_dir)
from data_transform.load_csv import load_csv
from data_transform.map_cimas_to_depth_multiple import map_cimas_create_columns
from data_transform.normalize_well_logs_by_cima import normalize_well_logs_by_cima
from data_transform.print_unique_values import print_unique_values
from data_transform.save_data_csv import save_data_csv

# Codigo para imprimir mas de una salida de la misma celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Creamos un DataFrame de trabajo

In [9]:
# Definimos la ruta de los registros a normalizar y sus cimas
path_well_logs = os.path.join(notebook_dir, "..", "data", "raw", "pijije_para_normalizar.csv")
path_well_tops = os.path.join(notebook_dir, "..", "data", "raw", "pijije_cimas_normalizar.csv")

# Cargamos la informacion de los pozos
well_logs_df = load_csv(path_well_logs)

# verificamos que no exista ningun valor negativo en md
well_logs_df = well_logs_df[well_logs_df["md"] >= 0]

# Cargamos la informacion de las cimas
well_tops_df = load_csv(path_well_tops)

# verificamos que no exista ningun valor negativo en md
well_logs_df = well_logs_df[well_logs_df["md"] >= 0]

# Cargamos la informacion de las cimas
well_tops_df = load_csv(path_well_tops)

# Borramos la cima "PT"
well_tops_df = well_tops_df[well_tops_df["surface"] != "PT"]

# Mapeamos las cimas con los registros de pozo 
pijije_df = map_cimas_create_columns(well_tops_df, well_logs_df, True)

# Eliminamos todos los valores nulos
pijije_df.dropna(inplace=True)

# Imprimimos todas las cimas presentes en el DataFrame pijije_df
print("="*150)
print_unique_values(pijije_df, "cima")
print("="*150)
print_unique_values(pijije_df, "wellname")
print("="*150)



Valores únicos en la columna 'cima': cnir, enct, dpst, olgn, sal_ter, eocn, pacn, mndz, snfl, agnv, crtcm, crti, ttnno, kmmgn
Valores únicos en la columna 'wellname': Pijije-101, Pijije-2, Pijije-25, Pijije-27


## Normalización

In [3]:
# Seleccionamos el DataFrame de interes
df = pijije_df

# Seleccionamos los datos de las cimas de vamos a normalizar para cada arena
cimas_interes = ["mndz", "snfl", "agnv", "crtcm", "crti", "ttnno", "kmmgn"]

# Seleccionamos el pozo clave
key_well = "Pijije-2"

# Seleccionar las curvas a normalizar
curve_names = ["dtsm"]

# Seleccionamos los cuantiles para la normalización
quanti_low = 0.05
quanti_high = 0.95

# Normalizamos las curvas por cima
pijije_normalizado_list = normalize_well_logs_by_cima(df, cimas_interes, key_well, curve_names, quanti_low=0.05, quanti_high=0.95)

# Concatenar la lista de DataFrames en un solo DataFrame
pijije_normalizado_df = pd.concat(pijije_normalizado_list, ignore_index=True)

## Generación de CSV

In [6]:
# Generamos un archivo csv con las curvas normalizados para los pozos de interes
# Pozos de interes
normalized_wells = ["Pijije-25"]

# Guardamos la información de curvas normalizadas
save_data_csv(pijije_normalizado_df, normalized_wells, "dtsm_normalized.csv")